In [ ]:
import sys
import os

parent = os.path.dirname(os.path.realpath('../'))
sys.path.append(parent)

import numpy as np
import scipy
import tqdm
import open3d as o3d
import matplotlib.pyplot as plt
import glob

from core import *
from utils import phantom_builder
from utils import geometry
from utils import utils

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

overwrite_figures = True
figure_path = './outputs'
if not os.path.exists(figure_path):
    os.makedirs(figure_path)

rc = {"mathtext.fontset" : "stix"}
plt.rcParams.update(rc)

In [ ]:
test_phantom = phantom.Phantom(
            voxel_dims = (0.5e-3,0.5e-3,0.5e-3),
            matrix_dims = (256,256,64),
            baseline = (1500, 1000),
            seed = 5678,)

water = tissue.Tissue(name='water', c=1500, rho=1000, sigma=0, scale=0.001, label=0)
wire = tissue.Tissue(name='wire', c=2400, rho=1500, sigma=0, scale=0.001, label=1)

pos = 0.016
test_phantom.add_tissue(water, mask=np.ones(test_phantom.matrix_dims))

wire_mask = np.zeros(test_phantom.matrix_dims)
for i in range(4,13): 
    for j in range(1,13): 
        wire_mask[int(0.01/test_phantom.voxel_dims[0]*i - 2),int(0.01/test_phantom.voxel_dims[0]*j - 2),:] = 1
    
test_phantom.add_tissue(wire, mask=wire_mask)
test_phantom.set_default_tissue('water')
test = test_phantom.get_complete()

In [ ]:
plt.imshow(test[0,:,:,test_phantom.matrix_dims[2]//2])

plt.gca().set_xticks(np.linspace(0, test_phantom.matrix_dims[1],9))
plt.gca().set_yticks(np.linspace(0, test_phantom.matrix_dims[0],9))
plt.gca().grid(color='w', linestyle='-', linewidth=0.1)

In [ ]:
num_transducers = 1
transducers = [transducer.Planewave(max_frequency=2e6,
                                    elements = 256, 
                                    width = 40e-3,
                                    height =  10e-3,
                                    sensor_sampling_scheme = 'not_centroid', 
                                    sweep = np.pi/2,
                                    ray_num = 25,
                                    imaging_ndims = 2,
                                    focus_elevation = 100e-3,
                                    ) for i in range(num_transducers)]


for t in transducers:
    t.make_sensor_coords(test_phantom.baseline[0])

In [ ]:
cont_transducer_set = transducer_set.TransducerSet(transducers, seed=8888)
cont_transducer_set.assign_pose(0, geometry.Transform([0,0,0], [-0.05,0,0]))

cont_transducer_set.plot_transducer_fovs(scale=0.1)
cont_transducer_set.plot_transducer_coords(scale=0.1)

In [ ]:
test_sensor = sensor.Sensor(transducer_set=cont_transducer_set, aperture_type='extended_aperture')


In [ ]:
simprops = simulation.SimProperties(
                grid_size   = (100e-3,110e-3,20e-3),
                PML_size    = (16,8,8),
                PML_alpha   = 2,
                t_end       = 12e-5,           # [s]
                bona        = 6,               # parameter b/a determining degree of nonlinear acoustic effects
                alpha_coeff = 0.5, 	           # [dB/(MHz^y cm)]
                alpha_power = 1.5,
                )

In [ ]:
test_experiment = experiment.Experiment(
                 simulation_path = 'single',
                 sim_properties  = simprops,
                 phantom         = test_phantom,
                 transducer_set  = cont_transducer_set,
                 sensor          = test_sensor,
                 nodes           = 1,
                 results         = None,
                 indices         = None,
                 workers         = 3,
                 additional_keys = []
                 )

test_experiment.save()

In [ ]:
gm = test_experiment.visualize_sensor_mask(index=[slice(0, -1, 1), slice(0, -1, 1), test_phantom.matrix_dims[2]//2])
plt.gca().set_title('aperture')
plt.gca().set_xticks([-1,256])
plt.gca().set_xticklabels(['0','0.128'])
plt.gca().set_yticks([0,256])
plt.gca().set_yticklabels(['0.128','0'])
plt.xlabel('x (m)')
plt.ylabel('y (m)')
plt.savefig(f'{figure_path}/singletransducer_aperture.svg', dpi=300, bbox_inches="tight")

In [ ]:
test_experiment = experiment.Experiment.load('single')
test_experiment.run(dry=False)
test_experiment.run(repeat=False)

# Reconstruction

In [ ]:
test_experiment = experiment.Experiment.load('single')
test_experiment.run(dry=True)
test_experiment.add_results()
test_reconstruction = reconstruction.Compounding(experiment=test_experiment)

In [ ]:
images1 = test_reconstruction.compound(workers=9, resolution_multiplier=2, combine=False, attenuation_factor=4)

In [ ]:
image1 = np.sum(images1, axis=0)/len(images1)

In [ ]:
from scipy.interpolate import NearestNDInterpolator
msensor = test_experiment.get_sensor_mask()
x = []
y = []
z = []
for i in tqdm.tqdm(range(msensor.shape[0])):
    for j in range(msensor.shape[1]):
        x.append(i)
        y.append(j)
        z.append(msensor[i,j,:].sum(axis=-1))
X = np.linspace(0, msensor.shape[0], image1.shape[0])
Y = np.linspace(0, msensor.shape[1], image1.shape[1])
X, Y = np.meshgrid(X, Y, indexing='ij')  # 2D grid for interpolation
interp = NearestNDInterpolator(list(zip(x, y)), z)
Z = interp(X, Y)
sensor_mask = Z

In [ ]:
psf = 20 * np.log10(image1 / np.max(image1))
image = psf[:,:,psf.shape[2]//2]

fig, ax = plt.subplots()
contour_cmap = plt.cm.get_cmap('plasma')

im = ax.imshow(image, cmap='gray', vmin=-35, vmax=0)
ax.imshow(sensor_mask, cmap='cool_r', alpha=(sensor_mask>0)*1.0)
# CS = ax.contour(image, cmap=contour_cmap, linewidths=0.5, levels = [-25,-20,-15,-10,-5,0])

ax.set_title('single, $\mathrm{\mathbb{P}}(E)=1$')

ax.set_xticks([])
ax.set_yticks([])
if overwrite_figures:
    plt.savefig(f'{figure_path}/single_noap.png', dpi=300, bbox_inches="tight")

In [ ]:
from core.analytic_wave import planewave_pmax

In [ ]:
test_experiment = experiment.Experiment.load('single')
test_experiment.run(dry=True)
test_experiment.add_results()
test_reconstruction = reconstruction.Compounding(experiment=test_experiment)

In [ ]:
pmax = planewave_pmax(resolution=0.0005, 
                    matrix_size=(256,256), 
                    L=test_reconstruction.transducer_set[0][0].width, 
                    f=test_reconstruction.transducer_set[0][0].max_frequency, 
                    m=[-2,-1,0,1,2], 
                    w=test_reconstruction.transducer_set[0][0].width / test_reconstruction.transducer_set[0][0].get_num_elements(), 
                    c_0=1500, 
                    rho_0=1000, 
                    v_0=5e-8, 
                    tau=1, 
                    t=np.linspace(0, 10, 100), 
                    omega_c=1)

plt.imshow(pmax, cmap='hot')
plt.yticks([0,255])
plt.xticks([0,128,255])
plt.gca().set_yticklabels([f'{0}',f'{256*0.0005}'])
plt.gca().set_xticklabels([f'{-128*0.0005}', f'{0}', f'{128*0.0005}'])
plt.xlabel('x (m)')
plt.ylabel('y (m)')

plt.savefig(f'{figure_path}/singletransducer_pressure_field.png', dpi=300, bbox_inches="tight")

In [ ]:
images3 = test_reconstruction.compound(workers=9, resolution_multiplier=2, combine=False, pressure_field=pmax, pressure_field_resolution=0.0005, attenuation_factor=4)

In [ ]:
image3 = np.sum(images3, axis=0)/len(images3)

In [ ]:
from scipy.interpolate import NearestNDInterpolator
msensor = test_experiment.get_sensor_mask()
x = []
y = []
z = []
for i in tqdm.tqdm(range(msensor.shape[0])):
    for j in range(msensor.shape[1]):
        x.append(i)
        y.append(j)
        z.append(msensor[i,j,:].sum(axis=-1))
X = np.linspace(0, msensor.shape[0], image1.shape[0])
Y = np.linspace(0, msensor.shape[1], image1.shape[1])
X, Y = np.meshgrid(X, Y, indexing='ij')  # 2D grid for interpolation
interp = NearestNDInterpolator(list(zip(x, y)), z)
Z = interp(X, Y)
sensor_mask = Z

In [ ]:
psf = 20 * np.log10(image3 / np.max(image3))
image = psf[:,:,psf.shape[2]//2]

fig, ax = plt.subplots()
contour_cmap = plt.cm.get_cmap('plasma')

im = ax.imshow(image, cmap='gray', vmin=-35, vmax=0)
ax.imshow(sensor_mask, cmap='cool_r', alpha=(sensor_mask>0)*1.0)
# CS = ax.contour(image, cmap=contour_cmap, linewidths=0.5, levels = [-25,-20,-15,-10,-5,0])

plt.colorbar(im, cax=ax.inset_axes([1.035, 0, 0.04, 1]), label='dB', ticks=[-35,-25,-20,-15,-10,-5,0])
# proxy = [plt.Rectangle((0,0),1,1,fc = pc.get_edgecolor()[0]) for pc in CS.collections]
# plt.legend(proxy[::-1], ['','','','','','','',''][::-1], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1.138, 0.905), labelspacing = 2.8)

ax.set_title('single, $\mathrm{\mathbb{P}}(E)=P$')

ax.set_xticks([])
ax.set_yticks([])
if overwrite_figures:
    plt.savefig(f'{figure_path}/single_ap.png', dpi=300, bbox_inches="tight")

In [ ]:
res_horz_indices = np.where(np.sum(test_reconstruction.phantom.mask, axis=(0,2))>128)[0]
res_horz_indices = res_horz_indices * image1.shape[0] / test_reconstruction.phantom.matrix_dims[0]
res_vert_indices = np.where(np.sum(test_reconstruction.phantom.mask, axis=(1,2))>128)[0]
res_vert_indices = res_vert_indices * image1.shape[0] / test_reconstruction.phantom.matrix_dims[0]

In [ ]:
plt.rcParams.update({'font.size': 14})

In [ ]:
cmap3 = plt.cm.get_cmap('YlGn', 9)

# near resolution horizontal
fig, ax = plt.subplots(2,1, figsize=(7,4), tight_layout=True)
for xc in res_horz_indices:
    ax[0].vlines(x=xc-3, ymin=-150,ymax=0, linewidth=0.5, color='k', linestyle='--')
    ax[1].vlines(x=xc-3, ymin=-150,ymax=0, linewidth=0.5, color='k', linestyle='--')

for row,index in enumerate(res_vert_indices[::-1]):
    ax[0].plot(20*np.log(image1[int(index)] / np.max(image1[int(index)])), linewidth = 0.8, c=cmap3(row), label='not compensated')
    ax[1].plot(20*np.log(image3[int(index)] / np.max(image3[int(index)])), linewidth = 0.8, c=cmap3(row), label='excitation compensated')
ax[0].spines['top'].set_visible(False)
ax[0].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].spines['right'].set_visible(False)
ax[0].set_ylabel('intensity (dB)')
ax[1].set_ylabel('intensity (dB)')
ax[1].set_xticks([0,image1.shape[0]]) # 133 / 683 * 256 * 0.5
ax[1].set_xticklabels(['0','128'])
ax[0].set_xticks([]) # 133 / 683 * 256 * 0.5
ax[0].set_xticklabels([])
ax[1].set_xlabel('x (mm)')

In [ ]:
cmap3 = plt.cm.get_cmap('YlGn', 9)

# near resolution horizontal
fig, ax = plt.subplots(1,2, figsize=(10,3.5), tight_layout=True)
for xc in res_horz_indices:
    ax[0].vlines(x=xc-3, ymin=-140,ymax=0, linewidth=0.5, color='k', linestyle='--')
    ax[1].vlines(x=xc-3, ymin=-140,ymax=0, linewidth=0.5, color='k', linestyle='--')

for row,index in enumerate(res_vert_indices[::-1]):
    ax[0].plot(20*np.log(image1[int(index)] / np.max(image1[int(index)])), linewidth = 1.5, c=cmap3(row), label='sequential')
    ax[1].plot(20*np.log(image3[int(index)] / np.max(image3[int(index)])), linewidth = 1.5, c=cmap3(row), label='synchronous')

for a in ax.flatten():
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)
    a.spines['top'].set_visible(False)
    a.spines['right'].set_visible(False)
ax[0].set_ylabel('intensity (dB)')
# ax[1].set_ylabel('intensity (dB)')

# ax[0].set_xticks([])
# ax[1].set_xticks([])
# ax[0].set_xticklabels([])
ax[0].set_xticks([0,image3.shape[0]])
ax[0].set_xticklabels(['0','128'])
ax[0].set_xlabel('x (mm)')
ax[1].set_xticks([0,image3.shape[0]])
ax[1].set_xticklabels(['0','128'])
ax[1].set_xlabel('x (mm)')

# proxy = [plt.Rectangle((0,0),1,1,fc = cmap3(n)) for n in range(cmap3.N)]
# ax[0].legend(proxy[::-1], ['20 mm','','','','','','','100 mm'], frameon=False, handlelength=0.5, handleheight=1, fontsize=10, bbox_to_anchor=(1, 1.05), labelspacing = 0, title='depth\n')
proxy = [plt.Rectangle((0,0),1,1,fc = cmap3(n)) for n in range(cmap3.N)]
ax[1].legend(proxy[::-1], ['20 mm','','','','','','','100 mm'], frameon=False, handlelength=0.5, handleheight=1, fontsize=14, bbox_to_anchor=(1, 1.05), labelspacing = 0, title='depth\n')

plt.savefig(f'{figure_path}/grid_psf_2panel.svg', bbox_inches="tight")

In [ ]:
def compute_SNR(image, res_horz_indices, res_vert_indices, spread=13,):
    signal = np.zeros((res_vert_indices.shape[0], res_horz_indices.shape[0]))
    signal_points = []
    for i,m in enumerate(res_vert_indices):
        for j,n in enumerate(res_horz_indices):
            signal[i,j] = image[int(np.round(m)),int(np.round(n))]
            signal_points.append((n,m))
            
    noise = np.zeros((res_vert_indices.shape[0], res_horz_indices.shape[0]))
    noise_points = []
    for i,m in enumerate(res_vert_indices):
        for j,n in enumerate(res_horz_indices):
            noise[i,j] += image[int(np.round(m+spread)),int(np.round(n+spread))]
            noise[i,j] += image[int(np.round(m-spread)),int(np.round(n+spread))]
            noise[i,j] += image[int(np.round(m+spread)),int(np.round(n-spread))]
            noise[i,j] += image[int(np.round(m-spread)),int(np.round(n-spread))]
            noise_points.append((n+spread,m+spread))
            noise_points.append((n-spread,m+spread))
            noise_points.append((n+spread,m-spread))
            noise_points.append((n-spread,m-spread))
            
    signal_points = np.array(signal_points)
    noise_points = np.array(noise_points)
    
    noise = noise/4
    
    return signal, noise, signal_points, noise_points

In [ ]:
signal1, noise1, signal_points1, noise_points1 = compute_SNR(image1, res_horz_indices, res_vert_indices)
signal3, noise3, signal_points3, noise_points3 = compute_SNR(image3, res_horz_indices, res_vert_indices)

error1 = np.std(signal1/noise1, axis=1)
error3 = np.std(signal3/noise3, axis=1)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(4,4), tight_layout=True)

ax.plot(np.mean(signal1/noise1, axis=1), color=cmap3(6), linestyle = '-',  label='$\mathrm{\mathbb{P}}(E)=1$', linewidth=2)
ax.plot(np.mean(signal3/noise3, axis=1), color=cmap3(6), linestyle = '--', label='$\mathrm{\mathbb{P}}(E)=P$', linewidth=2)

ax.fill_between(np.arange(9), np.mean(signal1/noise1, axis=1)-error1, np.mean(signal1/noise1, axis=1)+error1, color=cmap3(6), alpha=0.15, linewidth=0)
ax.fill_between(np.arange(9), np.mean(signal3/noise3, axis=1)-error3, np.mean(signal3/noise3, axis=1)+error3, color=cmap3(6), alpha=0.15, linewidth=0)
ax.legend()

ax.set_xticks([0,8])
ax.set_xticklabels(['20','100'])
ax.set_xlabel('depth (mm)')
ax.set_ylabel('SNR')

plt.savefig(f'{figure_path}/singletransducer_SNR.svg', bbox_inches="tight")

In [ ]:
# plt.imshow(image, cmap='gray', vmin=-30, vmax=0)
# plt.scatter(signal_points1[:,0],signal_points1[:,1], c=[0.2,0.4,1], s=2)
# plt.scatter(noise_points1[:,0], noise_points1[:,1], c='orange', s=2)

# plt.savefig(f'{figure_path}/singletransducer_SNR_points.png', dpi=300, bbox_inches="tight")

plt.imshow(image, cmap='gray', vmin=-30, vmax=0)
plt.scatter(signal_points1[:,0],signal_points1[:,1], c=[0.2,0.4,1], s=2, label='signal')
plt.scatter(noise_points1[:,0], noise_points1[:,1], c='orange', s=2, label='noise')
plt.gca().set_xticks([])
plt.gca().set_yticks([])
legend = plt.legend(loc="upper right", edgecolor="white")
legend.get_frame().set_alpha(None)
legend.get_frame().set_facecolor((1, 1, 1, 1))
plt.savefig(f'{figure_path}/singletransducer_SNR_points.png', dpi=300, bbox_inches="tight")

In [ ]:
# cmap1 = plt.cm.get_cmap('GnBu', 9)
# cmap2 = plt.cm.get_cmap('OrRd', 9)

# # near resolution horizontal
# fig, ax = plt.subplots(2,2, figsize=(8,4), tight_layout=True)
# for xc in res_horz_indices:
#     ax[0,0].vlines(x=xc-3, ymin=-125,ymax=0, linewidth=0.5, color='k', linestyle='--')
#     ax[0,1].vlines(x=xc-3, ymin=-125,ymax=0, linewidth=0.5, color='k', linestyle='--')
#     ax[1,0].vlines(x=xc-3, ymin=-125,ymax=0, linewidth=0.5, color='k', linestyle='--')
#     ax[1,1].vlines(x=xc-3, ymin=-125,ymax=0, linewidth=0.5, color='k', linestyle='--')

# for row,index in enumerate(res_vert_indices[::-1]):
#     ax[0,0].plot(20*np.log(image1[int(index)] / np.max(image1[int(index)])), linewidth = 0.8, c=cmap1(row), label='sequential')
#     ax[0,1].plot(20*np.log(image3[int(index)] / np.max(image3[int(index)])), linewidth = 0.8, c=cmap1(row), label='synchronous')
#     ax[1,0].plot(20*np.log(image2[int(index)] / np.max(image2[int(index)])), linewidth = 0.8, c=cmap2(row), label='sequential')
#     ax[1,1].plot(20*np.log(image4[int(index)] / np.max(image4[int(index)])), linewidth = 0.8, c=cmap2(row), label='synchronous')

# for a in ax.flatten():
#     a.spines['top'].set_visible(False)
#     a.spines['right'].set_visible(False)
#     a.spines['top'].set_visible(False)
#     a.spines['right'].set_visible(False)
# ax[0,0].set_ylabel('intensity (dB)')
# ax[1,0].set_ylabel('intensity (dB)')

# ax[1,0].set_xticks([0,image3.shape[0]])
# ax[1,0].set_xticklabels(['0','128'])
# ax[1,1].set_xticks([0,image3.shape[0]])
# ax[1,1].set_xticklabels(['0','128'])
# ax[0,0].set_xticks([])
# ax[0,1].set_xticks([])
# ax[0,0].set_xticklabels([])
# ax[0,1].set_xticklabels([])
# ax[1,0].set_xlabel('x (mm)')
# ax[1,1].set_xlabel('x (mm)')

# proxy = [plt.Rectangle((0,0),1,1,fc = cmap1(n)) for n in range(cmap1.N)]
# ax[0,1].legend(proxy[::-1], ['20 mm','','','','','','','100 mm'], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1, 0.95), labelspacing = 0)
# proxy = [plt.Rectangle((0,0),1,1,fc = cmap2(n)) for n in range(cmap2.N)]
# ax[1,1].legend(proxy[::-1], ['20 mm','','','','','','','100 mm'], frameon=False, handlelength=1, handleheight=0.01, fontsize=10, bbox_to_anchor=(1, 0.95), labelspacing = 0)

# plt.savefig(f'grid_psf_4panel.svg', bbox_inches="tight")